# Creating a new Area of Interest
**Author** Robert Zinke, Simran Sangha, David Bekaert - Jet Propulsion Laboratory

This notebook demonstrates suggested practices for creating a new study area of interest (AOI) for Sentinel-1 acquisitions. This is done by browsing products available through the [Alaska Satellite Facility (ASF) Data Search Vertex](https://search.asf.alaska.edu/#/) site. Below, we demonstrate the AOI selection process using the **`ariaAOIassist.py`** function.

The process of creating a new AOI follows the general workflow:
1. Download metadata from ASF Vertex
2. Extract the data coverage using the ariaAOIassist.py function
3. Filter results and determine the best processing strategy that weighs InSAR coverage in time and space
4. Precisely define the AOI as a polygon in Google Earth, being certain to trim the polygon edges by approximately one burst or ~20 km

NOTE: Creating a new AOI is often an iterative process and particular attention should be paid to ensuring the final AOI bounds are within the area of coverage. The `ariaAOIassist.py` function cannot ensure the creation of an exact AOI that meets all criteria, but is designed the help the user in determining the best strategy for processing. Contact xxx@jpl.nasa.gov for assistance.

## 1. Download metadata from ASF Vertex

ARIA uses available Sentinel-1 processed data products. These products and their associated metadata are available through the [ASF Vertex](https://search.asf.alaska.edu/#/) webpage: **https://search.asf.alaska.edu/#/**

This interface allows the user to interactively search by numerous criteria.

### Search by area and date
The simplest means of filtering are to search by area and date. To search by area, simply click once within the map area to specify one corner of a bounding box, then move the mouse and click elsewhere to specify the opposite corner. The start and end dates can be specified in the main ribbon.

### Filter for SLC products and other criteria

Additional useful filters can be applied. For AOI creation, we strongly recommend specifying the File Type as **L1 Single Look Complex (SLC)** and the Beam Mode as **IW** (interferometric wideswath), as these are the data products used by ARIA. Other criteria such as the orbit direction (ascending/descending), path number, and frame numbers can also be specified.

<img src="./support_docs/ASFfilters.png" alt="ASFfilters" width="700">
<center><b>Fig. 1</b> Additional filters for data products. We recommend specifying the File Type as SLC and the Beam Mode as IW.</center>

If desired, RAW frames can also be *viewed* using the **`ariaAOIassist.py`** function, as described below. Because RAW frames cannot be processed by ARIA, we generally exclude these from our workflow.

## Initial setup

In [1]:
# Import modules
import os
from glob import glob

# Define directories
work_dir = os.path.abspath(os.getcwd())
tutorial_home_dir = os.path.abspath(os.getcwd())

# Create work directory if it does not yet exist
work_dir = os.path.join(work_dir,'work')
if not os.path.exists(work_dir):
    os.mkdir(work_dir)

# Verify ARIA-tools is installed correctly
try:
    import ARIAtools.shapefile_util as shputil
except:
    raise Exception('ARIA-tools is missing from your PYTHONPATH')

os.chdir(work_dir)

### Download metadata
We will first start with an area over central Tibet, between longitudes 87-92E and latitudes 33-40N.

For this example, simply copy the polygon below and paste it into the "Area of Interest" box in the ASF Vertex webpage.

`POLYGON((87 33,92 33,92 40,87 40,87 33))`

Further limit the acquisitions by specifying the end date as January 1st, 2017 (1/1/2017). Under the "Filters" menu, specify the File Type as "L1 Single Look Complex (SLC)" and the Beam Mode as "IW".

This should produce more than 1,000 results, the first 250 of which are shown.

<img src="./support_docs/ASF_Search_results.png" alt="Search results" width="700">
<center><b>Fig. 2</b>ASF Vertex search results from central Tibet</center>

The metadata for all these results can be downloaded in the form of a comma-separated value (.csv) file. Under the SEARCH box, and next to the Filters menu, the Vertex website provides the number of files currently listed (default = 250). Next to this number is a small arrow indicating a dropdown menu. Click that arrow and select the option for **More Files**, as shown below:

<img src="./support_docs/ASF_more_files.png" alt="MoreFiles" width="700">
<center><b>Fig. 3</b> Option for <b>More Files.</b></center>

Download the results and save them to your *work_dir* (i.e., `./work`).

The first part of the automatically generated name will be "*asf-datapool-results-*" and the second part will be the date and time at which you downloaded the data.

For this example use the filename *Examples.csv*

In practice, it is generally recommended to draw a larger bounding box in ASF Vertex than is required, as filters can later be applied to select frames by latitude or date, as shown below.

### Create metadata products

Once the data have been downloaded, the `ariaAOIassist.py` function can be applied to them. By default, this function sorts the frame metadata by track and generates products that can be used to help define your AOI and determine the optimal strategy for ARIA processing.

For each track, products include a plot of the frame center latitudes as a function of acquisition date, and a Google Earth .kml file with the frame boundaries.

## Extract metadata using ariaAOIassist.py

We now apply the `ariaAOIassist.py` function to the .csv file we downloaded.

Execute the default command, e.g., `ariaAOIassist.py -f asf-datapool-results-example.csv`

This might take a minute or two to complete.

In [2]:
# Check that data were downloaded and named for this purpose
if not os.path.exists(os.path.join(work_dir,'Examples.csv')):
    raise Exception('Please ensure that the .csv data were downloaded to the work_dir and renamed \"Examples.csv\""')

# Run ariaAOIassist.py
!ariaAOIassist.py -f Examples.csv

Generating outputs for track: 70
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Generating outputs for track: 41
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Generating outputs for track: 12
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Generating outputs for track: 143
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opa

For each track, this produces a plot of the frame centers as a function of latitude and date, saved as postscript (.eps) files. Additionally, .kml files are created.

List the contents of the work_dir to confirm.

In [3]:
# List frame centers plots
!ls *.eps

A114_lat_extents.eps A41_lat_extents.eps  D150_lat_extents.eps
A12_lat_extents.eps  A70_lat_extents.eps  D19_lat_extents.eps
A143_lat_extents.eps D121_lat_extents.eps D48_lat_extents.eps


In [4]:
# List kml files
!ls *.kml

trackA114_frames.kml trackA41_frames.kml  trackD150_frames.kml
trackA12_frames.kml  trackA70_frames.kml  trackD19_frames.kml
trackA143_frames.kml trackD121_frames.kml trackD48_frames.kml


Open some of the new files to your work_dir to see them. Compare them to the frame positions in ASF Vertex. Note that the file naming convention gives the track number (e.g., 12) and the track direction (A = ascending; D = descending).

## 3. Filtering results

Several options are available in the `ariaAOIassist.py` function that allow for easy filtering of results, without the need to download a new metadata file each time.

Use the `ariaAOIassist.py -h` command to view all of the options:

In [5]:
!ariaAOIassist.py -h

usage: ariaAOIassist.py [-h] -f IMGFILE [-w WORKDIR] [-t TRACKS]
                        [-l LATBOUNDS] [-s STARTDATE] [-e ENDDATE]
                        [-x EXCLUDEDATES] [--plot_raw]
                        [--flag_partial_coverage] [--remove_incomplete_dates]
                        [--approximate_AOI]

Preparing preliminary plot of frame extents. First go to the ASF search page,
push all SLCs over defined search area to cart, download CSV under the
metadata option, and pass the CSV through to this script with the -f flag.

optional arguments:
  -h, --help            show this help message and exit
  -f IMGFILE, --file IMGFILE
                        Full path to CSV file containing SLC frame metadata.
  -w WORKDIR, --workdir WORKDIR
                        Specify directory to deposit all outputs. Default is
                        local directory where script is launched.
  -t TRACKS, --tracks TRACKS
                        Include only specified track number in results. Can be


As shown, results can be filtered by track, latitude, or date. For example, we can look at only track 41, between October 20, 2014, and August 16, 2016. We will send the outputs to a separate directory to avoid overwriting the original results.

In [6]:
!ariaAOIassist.py -f Examples.csv -t 41 -s 20141020 -e 20160816 -w "./Tr41_filtered"

Generating outputs for track: 41
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Products generated.


An example of the lat centers plots is shown below (resolution reduced for display purposes).

<img src="./support_docs/A41_lat_extents.png" alt="LatCenters" width="700">
<center><b>Fig. 4</b> Filtered results for track 41 (Oct 2014 - Aug 2016).</center>

Note that some of the dates do not have frames for every latitude (see 2015-04-18). We can specify a flag to highlight dates that do not cover the entire track extent.

In [7]:
!ariaAOIassist.py -f Examples.csv -t 41 -s 20141020 -e 20160816 -w "./Tr41_filtered" --flag_partial_coverage

Generating outputs for track: 41
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
Products generated.


<img src="./support_docs/A41_lat_extents_partials-flagged.png" alt="Flags" width="700">
<center><b>Fig. 5</b> Filtered results for track 41 (Oct 2014 - Aug 2016) - partial coverage flagged.</center>

If a latitude range is specified (using the `-l` option), dates that do not cover the full latitude range will be flagged as well.

For instance, `ariaAOIassist.py -f Examples.csv -t 41 -s 20141020 -e 20160816 -w "./Tr41_filtered" --flag_partial_coverage -l '33 41.6'`

<img src="./support_docs/A41_lat_extents_latbounds.png" alt="LatBounds" width="700">
<center><b>Fig. 6</b> Filtered results for track 41 (Oct 2014 - Aug 2016) - partial coverage flagged, latitude bounds specified.</center>

If desired, the user can specify which dates to exclude from further analysis using the `-x` option.

`ariaAOIassist.py -f Examples.csv -t 41 -s 20141020 -e 20160816 -w "./Tr41_filtered" --flag_partial_coverage -l '33 41.6' -x 20150418`

<img src="./support_docs/A41_lat_extents_dateExcl.png" alt="20150418 excluded" width="700">
<center><b>Fig. 6</b> Filtered results for track 41 (Oct 2014 - Aug 2016) - partial coverage flagged, latitude bounds specified. Note April 18, 2015 is manually excluded.</center>

If many dates are to be excluded, these can be listed in quotes (e.g., `-x '20151214 20160107'`, or in a text file (with extension .txt) and passed using the `-x` option.

*Excluded dates will NOT be included in the .kml*
This is designed to ease in the process of selecting a precise AOI, shown below. If one does not wish for incomplete dates to be included in the .kml or any further analyses, they may pass the `--remove_incomplete_dates` option.

`ariaAOIassist.py -f Examples.csv -t 41 -s 20141020 -e 20160816 -w "./Tr41_filtered" -l '33 41.6' --flag_partial_coverage --remove_incomplete_dates`

<img src="./support_docs/A41_lat_extents_incompletes-removed.png" alt="Incompletes removed" width="700">
<center><b>Fig. 7</b> Filtered results for track 41 (Oct 2014 - Aug 2016) - incomplete dates removed.</center>

## 4. AOI specification

The final step is to draw the AOI in Google Earth. Ultimately, this step requires the user the look closely at the kml file generated and draw a single polygon (rectangle) best approximating the shape of the data coverage.

**Some rules to keep in mind:**
* Dates for which data do not fully cover the polygon are automatically excluded
* The AOI must be smaller than the frame outline by 20 km, because sometimes the center swath burst exists but the bursts on either side do not. This will cause your AOI to fail the area threshold.

### Manually create AOI

You can create an AOI manually by opening the .kml file in Google Earth and drawing a polygon over the valid, overlapping frames.

First, open the kml Google Earth. Adjust the color settings by right clicking on the layer and selecting the "Get Info" option, then the "Style, Color" tab.

Here, I set the colors to semi-transparent so that the frame boundaries are visible.

<img src="./support_docs/GE_frames.png" width="700">
<center><b>Fig. 6</b> Frames for track 41 that meet all spatial and temporal requirements, shown in Google Earth.</center>

Draw an AOI by selecting the polygon tool and tracing over the area of overlap of all valid frames.

<img src="./support_docs/GE_manual_AOI_overview.png" width="700">
<center><b>Fig. 8</b> Google Earth polygon showing area of overlap.</center>

Then, zoom in to the AOI boundaries. Use the measuring tool to edit the polygons so that they are 20 km, or approximately one frame boundary from the northern and southern edges of the track boundaries. 

This is illustrated below:

<img src="./support_docs/GE_manual_20km_trim.png" width="700">
<center><b>Fig. 8</b> Trim approximately 20 km from northern and southern track boundaries.</center>

This ensures that frames will not be rejected due to lack of coverage.

### Suggested AOI (semi-automated)

`ariaAOIassist.py` has the ability to automatically suggest an AOI based on the overlap of all non-excluded frames. To use the feature, pass the `--approximate_AOI` option. 

**Be careful**--the approximate_AOI option in no way guarantees the validity of the suggested AOI. This function simply takes the union of all frames is calculated for each date. Then, the intersection of all swaths is taken. It attempts to remove ~20 km from the northern and southern edges of the polygon, but this is not always possible. Here again, the user must ensure that the polygon is within the limits of the area of data coverage.

It is often best to use the `approximate_AOI` option along with other filters, such as latitude bounds. Either exclude dates manually, or specify the `--remove_partial_dates` option. When doing so, the user is encouraged to refer back to the "lat_extents.eps" file to see which dates were automatically excluded.

<img src="./support_docs/GE_auto_AOI.png" width="700">
<center><b>Fig. 9</b> Automatically generated AOI. The removal of ~20 km of frame was achieved successfully for the southern boundary, but not for the northern. The user must manually ensure the appropriate distance from the frame boundary.</center>